<a href="https://colab.research.google.com/github/niteshctrl/sentiment_analysis/blob/main/1_Amazon_Reviews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Connecting GDrive and  Downloading data from Kaggle

In [ ]:
# Connecting Gdrive

from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [ ]:
# Upload Kaggle API Key

from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"nitesh1612","key":"7e13cc5de309e312f06e5744437ad7a1"}'}

In [ ]:
# Install Kaggle Library and auth permission

!pip install -q kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!ls ~/.kaggle
!chmod 600 /root/.kaggle/kaggle.json  # set permission

kaggle.json


In [ ]:
# Data Download from Kaggle

!kaggle datasets download -d 'snap/amazon-fine-food-reviews' -p '/content/gdrive/My Drive/colab/amazon'

 95% 229M/242M [00:01<00:00, 145MB/s]
100% 242M/242M [00:02<00:00, 126MB/s]


In [ ]:
# UNZIP downloadeded dataset

!unzip -q /content/gdrive/My\ Drive/colab/amazon/amazon-fine-food-reviews.zip -d /content/gdrive/My\ Drive/colab/amazon  #unzip data in train/

mkdir: cannot create directory ‘train’: File exists


# Amazon Fine Food Reviews Sentiment Analysis

### Data Description
This dataset consists of reviews of fine foods from amazon. The data span a period of more than 10 years, including all ~500,000 reviews up to October 2012. Reviews include product and user information, ratings, and a plain text review.

Data includes:

- Reviews from Oct 1999 - Oct 2012
- 568,454 reviews
- 256,059 users
- 74,258 products
- 260 users with > 50 reviews

### Objective

Given a review, determine whether a review is positive(4 or 5 star) or negative(1 or 2 star)


In [ ]:
# Import libraries

import sqlite3
import pandas as pd
import numpy as np
import scipy.sparse
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import BernoulliNB

# Text Preprocessing
import re
import string
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer

In [ ]:
y = np.load('/content/gdrive/My Drive/amazon/labels.npy')
final_bow = scipy.sparse.load_npz('/content/gdrive/My Drive/amazon/bow.npz')
final_tf_idf = scipy.sparse.load_npz('/content/gdrive/My Drive/amazon/tfidf.npz')
avg_w2v = np.load('/content/gdrive/My Drive/amazon/w2vec_sentences.npy')

In [ ]:
# Data Splitting

X_1, X_test, y_1, y_test = train_test_split(final_bow, y, test_size=0.1, random_state = 1)
X_tr, X_cv, y_tr, y_cv = train_test_split(X_1, y_1, test_size = 0.11, random_state = 1)

 ### Loading the data from SQLite File

In [ ]:
# Using SQLite Table to read data.
con = sqlite3.connect('/content/gdrive/My Drive/colab/amazon/database.sqlite') #Connection Creation

# Filtering only positive and negative reviews and ignoring neutral ones(Score = 3)
filtered_data = pd.read_sql_query("SELECT * FROM Reviews \
                WHERE Score !=3", con)

# Replace 5-class scores with 2-class constants 'postive'(Score=4,5) and 'negative'(Score=1,2) 
def partition(x):
    if x < 3:
        return 'negative'
    return 'positive'

actualScore = filtered_data['Score']
positiveNegative = actualScore.map(partition)
filtered_data['Score'] = positiveNegative

OperationalError: ignored

In [ ]:
print(filtered_data.shape)
filtered_data.head()

(525814, 10)


,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,positive,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,negative,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,positive,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,negative,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,positive,1350777600,Great taffy,Great taffy at a great price. There was a wid...


### Data Cleaning : Deduplication

In [ ]:
display = pd.read_sql_query("""SELECT * FROM Reviews WHERE 
                            Score !=3 AND UserId = "AR5J8UI46CURR" 
                            ORDER BY ProductID""", con)
display

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,78445,B000HDL1RQ,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...
1,138317,B000HDOPYC,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...
2,138277,B000HDOPYM,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...
3,73791,B000HDOPZG,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...
4,155049,B000PAQ75C,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...


#### Observation

It seems to be the same review has multiplied over different flavours of the same product. So we need to remove and keep just one.

In [ ]:
# Sorting data by ProductID in ascending order

sorted_data = filtered_data.sort_values('ProductId', axis=0, ascending=True)

In [ ]:
# Dropping the duplicates

final = sorted_data.drop_duplicates(subset={"UserId","ProfileName","Time","Text"}, keep='first', inplace=False)
final.shape

(364173, 10)

In [ ]:
# What %age of data is still left

final.size/filtered_data.size * 100

69.25890143662969

In [ ]:
display = pd.read_sql_query("""SELECT * FROM Reviews WHERE
            Score !=3 AND Id=44737 OR Id=64422 ORDER BY
            ProductId""", con)
display

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,64422,B000MIDROQ,A161DK06JJMCYF,"J. E. Stephens ""Jeanne""",3,1,5,1224892800,Bought This for My Son at College,My son loves spaghetti so I didn't hesitate or...
1,44737,B001EQ55RW,A2V0I904FH7ABY,Ram,3,2,4,1212883200,Pure cocoa taste with crunchy almonds inside,It was almost a 'love at first bite' - the per...


In [ ]:
# HelpfulnessNumerator is '#Users who found it helpful'
# HelpfulnessDenominator is '#Users who rated it useful or not'
# Therefore, always Numerator <= Denominator and discard the reviews which doesn't hold true.

final = final[final.HelpfulnessNumerator<=final.HelpfulnessDenominator]

In [ ]:
print(final.shape)

# Positive and Negative reviews left in our dataset
final['Score'].value_counts()

(364171, 10)


positive    307061
negative     57110
Name: Score, dtype: int64

#### Text Preprocessing
(StopWords removal, Stemming, Lemmitization)

Steps:
1. Remove HTML Tags
2. Remove Punctuations and Special Characters
3. Check the word is english and not alphanumeric
4. Check if the length of the word is greater than 2
5. Convert to lowercase
6. Remove Stopwords
7. Snowball stemming the word

Collect the words used to describe positive and negative reviews.

In [ ]:
stop = set(stopwords.words('english')) #Set of StopWords in English
sno = nltk.stem.SnowballStemmer('english') #Initialising the Snowball Stemmer

def cleanhtml(sentence):
    cleanr = re.compile('<.*?')
    cleantext = re.sub(cleanr, ' ', sentence) # re.sub = Substitute
    return cleantext

def cleanpunc(sentence):
    cleaned = re.sub(r'[?|!|\'|"|#]',r'', sentence) #Replaced with empty string
    cleaned = re.sub(r'[.|,|)|(|\|/]',r' ', cleaned) # Replaced with space
    return cleaned

print(sno.stem('tasty')) # Stem of the word 'Tasty'

tasti


In [ ]:
# Steps to implement checks mentioned in the pre-processing

i = 0
strl = ' '
final_string = []
all_positive_words = [] # Words from positive reviews
all_negative_words = []
s=''
for sent in final['Text'].values:
    filtered_sentence = []
    sent = cleanhtml(sent) # Remove HTML Tags
    for words in sent.split():
        for cleaned_words in cleanpunc(words).split():
            if(cleaned_words.isalpha()) & (len(cleaned_words) > 2):
                if(cleaned_words.lower() not in stop):
                    s = sno.stem(cleaned_words.lower()).encode('utf8')
                    filtered_sentence.append(s)
                    if final['Score'].values[i] == 'positive':
                        all_positive_words.append(s)
                    elif final['Score'].values[i] == 'negative':
                        all_negative_words.append(s)
                else:
                    continue
            else:
                continue
    strl = b" ".join(filtered_sentence) # List of words to String
    final_string.append(strl) # List of all String Reviews
    i += 1

#### Storing the Cleaned Reviews

In [ ]:
# Adding a Column of CleanedText to main DataFrame

final['CleanedText'] = final_string

In [ ]:
# Store final table into SQlLite file named 'final.sqlite'

conn = sqlite3.connect('/content/gdrive/My Drive/colab/amazon/final.sqlite')
c = conn.cursor()
conn.text_factory = str
final.to_sql('Reviews', conn, schema=None, if_exists='replace')

In [ ]:
# Storing true labels in variable 'y' and saving CleanedText in variable cleaned_text

cleaned_text = list(final['CleanedText'])
y = final['Score'].replace('positive', 1)
y = y.replace('negative',0)

In [ ]:
y_np = np.asarray(y, dtype=np.int8)

In [ ]:
y_np[0:20]

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0],
      dtype=int8)

In [ ]:
np.save('/content/gdrive/My Drive/amazon/labels', y_np)

#### Loading the cleaned reviews from the stored file

In [ ]:
# Loading data from the saved SQLite file

con = sqlite3.connect('/content/gdrive/My Drive/amazon/final.sqlite') #Connection Creation
final = pd.read_sql_query("SELECT * FROM Reviews", con)

In [ ]:
y = np.load('/content/gdrive/My Drive/amazon/labels.npy')

## Featurization

### Bag of Words(BoW)

In [ ]:
# BoW

count_vect = CountVectorizer()
final_bow = count_vect.fit_transform(final['Text'].values)

In [ ]:
print(type(final_bow))
final_bow.get_shape()

<class 'scipy.sparse.csr.csr_matrix'>


(364171, 115281)

In [ ]:
scipy.sparse.save_npz('/content/gdrive/My Drive/amazon/bow', final_bow)

In [ ]:
final_bow = scipy.sparse.load_npz('/content/gdrive/My Drive/amazon/bow.npz')

### Bi-Grams and n-Grams

In [ ]:
freq_dist_positive = nltk.FreqDist(all_positive_words)
freq_dist_negative = nltk.FreqDist(all_negative_words)
print("Most common Positive words :",freq_dist_positive.most_common(20))
print("\nMost common Negative words :",freq_dist_negative.most_common(20))

Most common Positive words : [(b'like', 139150), (b'tast', 128631), (b'good', 112216), (b'flavor', 109473), (b'love', 107034), (b'use', 103627), (b'great', 102818), (b'product', 99504), (b'one', 95360), (b'tri', 86237), (b'tea', 83824), (b'coffe', 78610), (b'make', 74835), (b'get', 71962), (b'food', 64752), (b'amazon', 57832), (b'would', 55297), (b'time', 55225), (b'buy', 53903), (b'realli', 52569)]

Most common Negative words : [(b'tast', 34489), (b'like', 32284), (b'product', 29504), (b'one', 20420), (b'flavor', 19561), (b'would', 17901), (b'tri', 17676), (b'use', 15275), (b'good', 14977), (b'coffe', 14677), (b'get', 13758), (b'buy', 13690), (b'order', 12846), (b'food', 12742), (b'dont', 11683), (b'tea', 11657), (b'amazon', 11258), (b'even', 10983), (b'box', 10841), (b'make', 9816)]


In [ ]:
# bi-gram, tri-gram and n-gram
# Removing stop words like "not" should be avoided before building n-grams

count_vect = CountVectorizer(ngram_range=(1,2)) #Includes Unigrams and Bigrams
final_bigram_counts = count_vect.fit_transform(final['Text'].values)

In [ ]:
final_bigram_counts.get_shape()

(364171, 2910192)

### TF-IDF

In [ ]:
tf_idf_vect = TfidfVectorizer(ngram_range=(1,1))
final_tf_idf = tf_idf_vect.fit_transform(final['Text'].values)

In [ ]:
scipy.sparse.save_npz('/content/gdrive/My Drive/amazon/tfidf', final_tf_idf)

In [ ]:
final_tf_idf = scipy.sparse.load_npz('/content/gdrive/My Drive/amazon/tfidf.npz')

In [ ]:
final_tf_idf.get_shape()

(364171, 115281)

In [ ]:
features = tf_idf_vect.get_feature_names()
len(features)

In [ ]:
features[100000:100010]

In [ ]:
print(final_tf_idf[3,:].toarray()[0])

In [ ]:
# Top TF-IDF Features

def top_tfidf_feats(row, features, top_n):
    topn_ids = np.argsort(row)[::-1][:top_n]
    top_feats = [(features[i], row[i]) for i in topn_ids]
    df = pd.DataFrame(top_feats)
    df.columns = ['Feature', 'tfidf']
    return df

top_tfidf = top_tfidf_feats(final_tf_idf[1,:].toarray()[0], features,25)

In [ ]:
top_tfidf

### Word2Vec

In [ ]:
# UNZIP the "googlenews_vectors" file

!gunzip /content/gdrive/My\ Drive/colab/amazon/googlenews_vectors.bin.gz

In [ ]:
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
import pickle

# We use pretrained model by Google
# It's 3.3GB once loaded into memory
# Occupies 10GB RAM
# Contains all our corpus words as keys and model[word] as values
# Download 'Google-news-vectors-negative300.bin'

model = KeyedVectors.load_word2vec_format('/content/gdrive/My Drive/colab/amazon/googlenews_vectors.bin', binary=True)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:254: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [ ]:
model.wv['computer'] # We can get the vector for any word

In [ ]:
model.wv.similarity('woman','man')

In [ ]:
model.wv.most_similar('woman')

In [ ]:
model.wv.most_similar('tasti') # Some of the stemmed words may not have Word to Vectors depending on the training dataset

In [ ]:
model.wv.most_similar('tasty')

In [ ]:
# Training our own Word2Vec model using our own text corpus
import gensim
i = 0
list_of_sent = []
for sent in final['Text'].values:
    filtered_sentence = []
    sent = cleanhtml(sent)
    for w in sent.split():
        for cleaned_words in cleanpunc(w).split():
            if(cleaned_words.isalpha()):
                filtered_sentence.append(cleaned_words.lower())
            else:
                continue
    list_of_sent.append(filtered_sentence)

In [ ]:
print(final['Text'].values[0])
print("*****************************************************")
print(list_of_sent[0])

In [ ]:
# min_count is the min frequency that a word should appear for training
# size is the size(dimension) of the vector
# Workers is the number of cores to be used for training

w2v_model = gensim.models.Word2Vec(list_of_sent, min_count=5,size=50, workers=-1)

In [ ]:
words = list(model.wv.vocab)
print(len(words))

3000000


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """Entry point for launching an IPython kernel.


In [ ]:
w2v_model.wv.most_similar('tasty')

In [ ]:
w2v_model.wv.most_similar('like')

In [ ]:
count_vect_feat = count_vect.get_feature_names()
count_vect_feat.index('like')
print(count_vect_feat[64055])

In [ ]:
model.wv['women']

## KNN

### Brute Force

#### Bow

In [ ]:
# Data Splitting(This should actually be done before Featurization to avoid data leakage)

X_1, X_test, y_1, y_test = train_test_split(final_bow, y, test_size=0.2, random_state = 1)
X_tr, X_cv, y_tr, y_cv = train_test_split(X_1, y_1, test_size = 0.25, random_state = 1)

In [ ]:
# KNN on BOW

neigh = KNeighborsClassifier(algorithm = 'brute', n_neighbors = 5, n_jobs = -1)
neigh.fit(X_tr, y_tr)
neigh.predict(X_cv[0:10])

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [ ]:
acc_array = np.array([])

In [ ]:
for i in range(5,16,2):
    neigh = KNeighborsClassifier(n_neighbors = i, n_jobs=-1)
    neigh.fit(X_tr, y_tr)
    pred = neigh.predict(X_cv)
    acc = accuracy_score(y_cv, pred, normalize=True) * float(100)
    acc_array = np.append(acc_array, acc)
    print('\n CV Accuracy for k = %d is %f%%' %(i,acc))


 CV Accuracy for k = 5 is 85.127825%

 CV Accuracy for k = 7 is 85.270615%

 CV Accuracy for k = 9 is 85.234918%

 CV Accuracy for k = 11 is 85.126452%


ValueError: ignored

In [ ]:
acc_array

array([85.12782492, 85.27061537, 85.23491776, 85.12645193])

In [ ]:
for i in range(5,16,2):
    neigh = KNeighborsClassifier(n_neighbors = i, n_jobs=-1)
    neigh.fit(X_tr, y_tr)
    pred = neigh.predict(X_cv)
    acc = accuracy_score(y_cv, pred, normalize=True) * float(100)
    acc_array = np.append(acc_array, acc)
    print('\n CV Accuracy for k = %d is %d%%' %(i,acc))


 CV Accuracy for k = 19 is 84%

 CV Accuracy for k = 21 is 84%

 CV Accuracy for k = 23 is 84%

 CV Accuracy for k = 25 is 84%

 CV Accuracy for k = 27 is 84%

 CV Accuracy for k = 29 is 84%


In [ ]:
acc_array

array([84.93148804, 84.90952028, 84.88480655, 84.87656864, 84.82851416,
       84.81341132])

In [ ]:
neigh = KNeighborsClassifier(n_neighbors = 7, n_jobs=-1)
neigh.fit(X_tr, y_tr)
pred = neigh.predict(X_test)
acc = accuracy_score(y_test, pred, normalize=True) * float(100)
print('\nFinal Test Accuracy for k = 7 is %f%%' %(acc))


Final Test Accuracy for k = 7 is 85.136267%


#### TF-IDF

In [ ]:
# Data Splitting(This should actually be done before Featurization to avoid data leakage)

X_1, X_test, y_1, y_test = train_test_split(final_tf_idf, y, test_size=0.2, random_state = 1)
X_tr, X_cv, y_tr, y_cv = train_test_split(X_1, y_1, test_size = 0.25, random_state = 1)

In [ ]:
# KNN on tf-idf

neigh = KNeighborsClassifier(algorithm = 'brute', n_neighbors = 5, n_jobs = -1)
neigh.fit(X_tr, y_tr)
neigh.predict(X_cv[0:10])

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [ ]:
acc_array = np.array([])

In [ ]:
i = 1
while i < 30:
    neigh = KNeighborsClassifier(n_neighbors = i, n_jobs=-1)
    neigh.fit(X_tr, y_tr)
    pred = neigh.predict(X_cv)
    acc = accuracy_score(y_cv, pred, normalize=True) * float(100)
    acc_array = np.append(acc_array, acc)
    print('\n CV Accuracy for k = %d is %.4f%%' %(i,acc))
    i = i*2 + 1


 CV Accuracy for k = 1 is 84.2299%

 CV Accuracy for k = 3 is 85.5342%

 CV Accuracy for k = 7 is 86.2880%

 CV Accuracy for k = 15 is 86.2811%


In [ ]:
i = 30
while i < 200:
    neigh = KNeighborsClassifier(n_neighbors = i, n_jobs=-1)
    neigh.fit(X_tr, y_tr)
    pred = neigh.predict(X_cv)
    acc = accuracy_score(y_cv, pred, normalize=True) * float(100)
    acc_array = np.append(acc_array, acc)
    print('\n CV Accuracy for k = %d is %.4f%%' %(i,acc))
    i = i*2 + 1


 CV Accuracy for k = 30 is 86.1699%

 CV Accuracy for k = 61 is 85.6551%

 CV Accuracy for k = 123 is 85.3887%


In [ ]:
i = 7
while i < 15:
    neigh = KNeighborsClassifier(n_neighbors = i, n_jobs=-1)
    neigh.fit(X_tr, y_tr)
    pred = neigh.predict(X_cv)
    acc = accuracy_score(y_cv, pred, normalize=True) * float(100)
    acc_array = np.append(acc_array, acc)
    print('\n CV Accuracy for k = %d is %.4f%%' %(i,acc))
    i = i + 2


 CV Accuracy for k = 7 is 86.2880%

 CV Accuracy for k = 9 is 86.3058%

 CV Accuracy for k = 11 is 86.3127%

 CV Accuracy for k = 13 is 86.2770%


#### Avg W2Vec

In [ ]:
# Convert bytes to Strings of cleaned_text

new_cleaned = []
for each in cleaned_text:
  new_cleaned.append(each.decode('utf-8'))

In [ ]:
len(new_cleaned)

364171

In [ ]:
# avg word to vector function

def avg_w2vec(sentence):
    split_words =sentence.split()
    c = 0
    vector = np.zeros((1,300))
    for word in split_words:
        if word in model.vocab:
            vector += model[word].reshape(1,300)
            c = c + 1
    if c is not 0:
        return vector/c
    else:
        return np.zeros([1,300])

In [ ]:
X = np.zeros([364171,300])

In [ ]:
# Sentence to avg_word2vec

from tqdm import tqdm
import time

for i in tqdm(range(364171), ncols = 100, position=0, leave=True):
    X[i] = avg_w2vec(new_cleaned[i])

100%|████████████████████████████████████████████████████▉| 363953/364171 [01:19<00:00, 3804.02it/s]


100%|█████████████████████████████████████████████████████| 364171/364171 [01:19<00:00, 4586.63it/s]


In [ ]:
np.save('/content/gdrive/My Drive/colab/amazon/w2vec_sentences', X)

In [ ]:
X = np.load('/content/gdrive/My Drive/amazon/w2vec_sentences.npy')

In [ ]:
X.shape

(364171, 300)

In [ ]:
# Data Splitting

X_1, X_test, y_1, y_test = train_test_split(X, y, test_size=0.1, random_state = 1)
X_tr, X_cv, y_tr, y_cv = train_test_split(X_1, y_1, test_size = 0.11, random_state = 1)

In [ ]:
print(len(X_tr), len(X_cv), len(X_test))

291700 36053 36418


In [ ]:
neigh = KNeighborsClassifier(n_neighbors=7, n_jobs = -1, algorithm = 'brute')
neigh.fit(X_tr, y_tr)

KNeighborsClassifier(algorithm='brute', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=-1, n_neighbors=7, p=2,
                     weights='uniform')

In [ ]:
neigh.predict(X_cv[0:10])

array([0, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [ ]:
pred = neigh.predict(X_cv)
acc = accuracy_score(y_cv, pred, normalize=True) * float(100)

In [ ]:
print('\n CV Accuracy for k = 7 is %.3f%%' %(acc))


 CV Accuracy for k = 7 is 85.629%


In [ ]:
acc_array = []

In [ ]:
i = 1
while i < 130:
    neigh = KNeighborsClassifier(algorithm='brute', n_neighbors = i, n_jobs=-1)
    neigh.fit(X_tr, y_tr)
    pred = neigh.predict(X_cv)
    acc = accuracy_score(y_cv, pred, normalize=True) * float(100)
    print('\n CV Accuracy for k = %d is %.3f%%' %(i,acc))
    acc_array.append(acc)
    i = i*2 + 1


 CV Accuracy for k = 1 is 80.997%

 CV Accuracy for k = 3 is 84.492%

 CV Accuracy for k = 7 is 85.629%

 CV Accuracy for k = 15 is 85.635%

 CV Accuracy for k = 31 is 85.491%

 CV Accuracy for k = 63 is 85.241%

 CV Accuracy for k = 127 is 84.950%


In [ ]:
i = 11
while i < 24:
    if i is not 15:
        neigh = KNeighborsClassifier(algorithm='brute', n_neighbors = i, n_jobs=-1)
        neigh.fit(X_tr, y_tr)
        pred = neigh.predict(X_cv)
        acc = accuracy_score(y_cv, pred, normalize=True) * float(100)
        print('\n CV Accuracy for k = %d is %.3f%%' %(i,acc))
        acc_array.append(acc)
    i = i + 2


 CV Accuracy for k = 11 is 85.643%

 CV Accuracy for k = 13 is 85.752%


KeyboardInterrupt: ignored

In [ ]:
i = 17
while i < 24:
    neigh = KNeighborsClassifier(algorithm='brute', n_neighbors = i, n_jobs=-1)
    neigh.fit(X_tr, y_tr)
    pred = neigh.predict(X_cv)
    acc = accuracy_score(y_cv, pred, normalize=True) * float(100)
    print('\n CV Accuracy for k = %d is %.3f%%' %(i,acc))
    acc_array.append(acc)
    i = i + 2 


 CV Accuracy for k = 17 is 85.629%

 CV Accuracy for k = 19 is 85.610%

 CV Accuracy for k = 21 is 85.580%

 CV Accuracy for k = 23 is 85.535%


### KD-Tree

#### BOW

In [ ]:
neigh = KNeighborsClassifier(algorithm='kd_tree', n_neighbors = 5, n_jobs=-1)
neigh.fit(X_tr, y_tr)

KNeighborsClassifier(algorithm='kd_tree', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=-1, n_neighbors=5, p=2,
                     weights='uniform')

In [ ]:
pred = neigh.predict(X_cv)

In [ ]:
i = 1
while i < 130:
    neigh = KNeighborsClassifier(algorithm='kd_tree', n_neighbors = i, n_jobs=-1)
    neigh.fit(X_tr, y_tr)
    pred = neigh.predict(X_cv)
    acc = accuracy_score(y_cv, pred, normalize=True) * float(100)
    print('\n CV Accuracy for k = %d is %.3f%%' %(i,acc))
    acc_array.append(acc)
    i = i*2 + 1 

## Naive Bayes

In [ ]:
n_bayes = BernoulliNB()
n_bayes.fit(X_tr, y_tr)

BernoulliNB(alpha=1.0, binarize=0.0, class_prior=None, fit_prior=True)

In [ ]:
pred = n_bayes.predict(X_cv)

In [ ]:
acc = accuracy_score(y_cv, pred, normalize=True) * float(100)
print(acc)

87.57173847378972


In [ ]:
alpha = 0.0000001
while alpha < 1000:
  n_bayes = BernoulliNB(alpha)
  n_bayes.fit(X_tr, y_tr)
  pred = n_bayes.predict(X_cv)
  acc = accuracy_score(y_cv, pred, normalize=True) * float(100)
  print(acc)
  alpha *= 10

88.50811434220282
88.57813658456215
88.65502375264299
88.73740286130104
88.78957629678446
88.76486256418706
88.44083807013207
87.57173847378972
84.5484251860395
84.400142790455


## Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn import preprocessing

X_scaled = preprocessing.scale(X_tr)

In [ ]:
log_reg = LogisticRegression().fit(X_scaled, y_tr)

In [ ]:
X_cv_scaled = preprocessing.scale(X_cv)

In [ ]:
pred = log_reg.predict(X_cv_scaled)

In [ ]:
acc = accuracy_score(y_cv, pred, normalize=True) * float(100)
print(acc)

88.30055751255097


In [ ]:
regul = 0.0000001
while regul < 1000:
  log_reg = LogisticRegression(C = regul).fit(X_scaled, y_tr)
  log_reg.fit(X_scaled, y_tr)
  pred = log_reg.predict(X_cv_scaled)
  acc = accuracy_score(y_cv, pred, normalize=True) * float(100)
  print(acc)
  regul *= 10

84.19826366737858
84.19826366737858
85.01650348098633
87.70698693589992
88.23121515546558
88.2728205697168
88.29778381826755
88.30055751255097
88.30333120683437
88.30333120683437


## SGD

## Support Vector Machines

In [ ]:
from sklearn.svm import SVC

clf = SVC(gamma='auto')
clf.fit(X_tr, y_tr)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [ ]:
pred = clf.predict(X_cv)

In [ ]:
acc = accuracy_score(y_cv, pred, normalize=True) * float(100)
acc

84.21490583307907

## Decision Trees

In [ ]:
from sklearn.tree import DecisionTreeClassifier

clf = DecisionTreeClassifier(max_depth=7)
clf.fit(X_tr, y_tr)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=7, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [ ]:
pred = clf.predict(X_tr)
acc = accuracy_score(y_tr, pred, normalize=True) * float(100)
print(acc)
pred = clf.predict(X_cv)
acc = accuracy_score(y_cv, pred, normalize=True) * float(100)
acc

84.81556393555022


84.53665436995534

In [ ]:
clf.get_depth()

10

## Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

i = 10
while i < 100:
  i += 20
  clf = RandomForestClassifier(n_estimators = i, n_jobs = -1)
  clf.fit(X_tr, y_tr)
  print(i,'\t', clf.score(X_tr, y_tr) * 100, clf.score(X_cv, y_cv) * 100)

30 	 99.96503256770654 86.66685157961889
50 	 99.9955433664724 86.48101406263001
70 	 99.99862872814535 86.38948215127729
90 	 99.99931436407267 86.29517654564114
110 	 99.99897154610902 86.12043380578592


In [ ]:
i = 5
print("#DT      D_tr Acc          D_cv Acc")
while i < 25:
  i += 5
  clf = RandomForestClassifier(n_estimators = i, n_jobs = -1)
  clf.fit(X_tr, y_tr)
  print(i,'\t', clf.score(X_tr, y_tr) * 100, clf.score(X_cv, y_cv) * 100)

#DT      D_tr Acc          D_cv Acc
10 	 99.6314706890641 87.46567553324273
15 	 99.6482687692835 86.7556097966882
20 	 99.90058279053822 87.19385349346796
25 	 99.89784024682893 86.45050342551244


## GBDT

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

clf = GradientBoostingClassifier()
clf.fit(X_tr, y_tr)
print(clf.score(X_cv, y_cv))

0.8771253432446676
